# Object Detection API TFRecord Generation

This notebook generates TFRecords, that are needed to use custom datasets with the TensorFlow Object Detection API.
These TFRecords can then be used to configure the training and the validation of the gesture detection model.
The documentation can be found on the official [TensorFlow Object Detection API Respository](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md#conversion-script-outline).

The [TFRecord generation notebook by Dat Tran](https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py) for his Raccoon Dataset proved to be a valuable resource.
It provides an implementation of the TFRecord format that fits the goal of this notebook very well. 

Some lines of code like paths have to be adjusted for your case. All needed adjustments are marked with "Todo".

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple

## Helper methods

In [ ]:
def split(df, group):
    """ Groups same image names together.
    One image can contain multiple hands. With this method those hands are grouped together 
    and attached to one image object.
    """
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

## Main method for TFRecord creation

In [ ]:
def create_tf_example(group, path):
    """ Create the TFRecord.
    This method creates the TFRecord according to the input data. It specifies how the data looks like.
    """
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    # Todo: Change file format if needed. Alternative: b'png'
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        # Todo: Map class to label
        # Should you need multiple labels a dedicated label mapping file can be used.
        # See https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb
        classes_text.append('hand'.encode('utf8'))
        classes.append(1)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

## TFRecord writer methods
After executing the data preprocessing script all files can be found under your defined folder name.
A folder will be generated with the following structure:

<pre>
ego_lared_tiny/  
├── images/  
│   ├── aishwaryfist000000109.jpg  
│   ├── aishwaryfist000000611.jpg
│   ├── aishwaryfist000000782.jpg
│   ├── ...
│   ├── aishwaryfist000001356.jpg
│   └── ... 
├── train/  
│   ├── aishwaryfist000000109.jpg  
│   ├── aishwaryfist000000782.jpg
│   └── ...  
├── val/
│   ├── aishwaryfist000000611.jpg  
│   ├── aishwaryfist000001356.jpg
│   └── ...  
├── labels_all.csv
├── labels_train.csv
└── labels_val.csv
</pre>

Please have a look at the following cells to see how these folders have to placed inside your detection_training/ folder.

In [ ]:
# Write training TFRecord
# Todo: Change file paths
writer = tf.python_io.TFRecordWriter("/home/jetbot/Documents/detection_training/train.record")
path = "/home/jetbot/Documents/detection_training/images_train/" # Path to training images
examples = pd.read_csv("/home/jetbot/Documents/detection_training/labels_train.csv") # Path to training labels
grouped = split(examples, 'frame')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
print("done")

In [ ]:
# Write validation TFRecord
# Todo: Change file paths
writer = tf.python_io.TFRecordWriter("/home/jetbot/Documents/detection_training/val.record")
path = "/home/jetbot/Documents/detection_training/images_val/" # Path to validation images
examples = pd.read_csv("/home/jetbot/Documents/detection_training/labels_val.csv") # Path to validation labels
grouped = split(examples, 'frame')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
print("done")

## Output TFRecords to console for verification

Outputting the records to the console allows manual verification of their correctness. 

In [ ]:
# Write training record to console
# Todo: Change file path
i = 1
for example in tf.python_io.tf_record_iterator("/home/jetbot/Documents/detection_training/train.record"):
    example = tf.train.Example.FromString(example)
    print(example)
    
    if i % 3 == 0:
        break
    i = i + 1
    

In [ ]:
# Write validation record to console
# Todo: Change file path
i = 1
for example in tf.python_io.tf_record_iterator("/home/jetbot/Documents/detection_training/val.record"):
    example = tf.train.Example.FromString(example)
    print(example)
    
    if i % 3 == 0:
        break
    i = i + 1